Je vais étudier dans ce notebook différents modèles d'apprentissage supervisé sur la base de donnée que que j'ai construite dans l'analyse exploratoire précédente. 
Cette base de donnée est un échantillon de l'ensemble des données de l'année 2016. 

In [307]:
# Chargement des librairies utiles 

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse 
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
from sklearn import model_selection
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn import metrics

In [289]:
# Chargement des bases de données 
data = pd.read_csv("2016_sample.csv", low_memory=False, sep=',')

# Feature engineering

Je vais ici diviser mes variables en deux types : les variables numériques et les variables représentant des catégories. 

In [290]:
scalingDF = data[['DISTANCE']] # Numerical features
categDF = data[['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK',
                  'CARRIER_CODE','ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID',
                    'DEP_TIME', 'ARR_TIME' 
                     ]] # Categorical features


Je dois ensuite enconder mes variables pour faciliter l'apprentissage des modèles supervisés testés ultérieurement. 

In [291]:
encoder = OneHotEncoder()# Create encoder object

categDF_encoded = encoder.fit_transform(categDF)


In [296]:
scalingDF_sparse = sparse.csr_matrix(scalingDF)

In [297]:
X_final = sparse.hstack((scalingDF_sparse, categDF_encoded))

In [298]:
y_final = data['ARR_DELAY'].values

In [299]:
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final,test_size = 0.2,random_state = 0) # Do 80/20 split

Il faut maintenant récupérer les variables numériques pour les mettre sur la même échelle que les autres variables afin qu'elles ne les écrasent pas dans l'apprentissage. 

In [300]:
X_train_numerical = X_train[:, 0:1].toarray() 
X_test_numerical = X_test[:, 0:1].toarray()

In [301]:
scaler = StandardScaler() # create scaler object
scaler.fit(X_train_numerical) # fit with the training data ONLY
X_train_numerical = sparse.csr_matrix(scaler.transform(X_train_numerical)) # Transform the data and convert to sparse
X_test_numerical = sparse.csr_matrix(scaler.transform(X_test_numerical))

In [302]:
X_train[:, 0:1] = X_train_numerical
X_test[:, 0:1] = X_test_numerical

# Tests des différents modèles

Je vais maintenant tester plusieurs modèles avec en baseline une approche de regression naïve qui prend simplement la moyenne du data set. 

## Baseline

In [306]:
dummy = DummyRegressor(strategy='mean')
dummy.fit(X_train, y_train)

DummyRegressor(constant=None, quantile=None, strategy='mean')

In [368]:
# Prédiction sur le jeu de test
y_pred_dum = dummy.predict(X_test)

# Evaluate
print("RMSE : %.2f" % np.sqrt(metrics.mean_squared_error(y_test, y_pred_dum)))

RMSE : 42.51


## Régression linéaire 

In [315]:
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [369]:
# Prédiction sur le jeu de test
y_pred_lr = lr.predict(X_test)

# Evaluate
print("RMSE : %.2f" % np.sqrt(metrics.mean_squared_error(y_test, y_pred_lr)))

RMSE : 41.84


Il semble qu'un modèle de régression linéaire simple soit plus précis que la baseline car le RMSE est inférieur. Je vais essayer de mieux faire en utilisant la régularisation. 

## Ridge

Afin d'optimiser le modèle de regression Ridge, je vais d'abord effectuer une validation croisée afin d'obtenir la valeur optimale d'alpha pour laquelle le modèle est le plus précis

In [352]:
ridge = model_selection.GridSearchCV(Ridge(), cv=5,
                                      param_grid={"alpha": np.logspace(0, 3, 40)},
                                    verbose=10, n_jobs=-1, scoring='neg_mean_squared_error')

In [353]:
ridge.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] alpha=1.0 .......................................................
[CV] alpha=1.0 .......................................................
[CV] alpha=1.0 .......................................................
[CV] alpha=1.0 .......................................................
[CV] .............. alpha=1.0, score=-1746.188577282866, total=  12.9s
[CV] alpha=1.0 .......................................................
[CV] ............. alpha=1.0, score=-1761.9962556887713, total=  16.6s
[CV] alpha=1.1937766417144364 ........................................
[CV] ............. alpha=1.0, score=-1741.4571957816775, total=  19.9s
[CV] alpha=1.1937766417144364 ........................................
[CV] .............. alpha=1.0, score=-1660.091571398308, total=  22.1s
[CV] alpha=1.1937766417144364 ........................................
[CV] ............. alpha=1.0, score=-1772.5351573899452, total=  16.4s
[CV] alpha=1.19

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   30.2s


[CV]  alpha=1.1937766417144364, score=-1746.15777014089, total=  11.9s
[CV] alpha=1.1937766417144364 ........................................
[CV]  alpha=1.1937766417144364, score=-1741.4329607320449, total=  17.4s
[CV] alpha=1.425102670302998 .........................................
[CV]  alpha=1.1937766417144364, score=-1761.9641217348603, total=  14.1s
[CV] alpha=1.425102670302998 .........................................
[CV]  alpha=1.1937766417144364, score=-1772.4994410135737, total=  17.3s
[CV]  alpha=1.425102670302998, score=-1746.1217961869643, total=  12.5s
[CV] alpha=1.425102670302998 .........................................
[CV] alpha=1.425102670302998 .........................................


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   50.0s


[CV]  alpha=1.1937766417144364, score=-1660.061528674597, total=  21.5s
[CV] alpha=1.425102670302998 .........................................
[CV]  alpha=1.425102670302998, score=-1741.4049179277042, total=  17.4s
[CV] alpha=1.701254279852589 .........................................
[CV]  alpha=1.425102670302998, score=-1761.9271550275066, total=  14.0s
[CV] alpha=1.701254279852589 .........................................
[CV]  alpha=1.425102670302998, score=-1772.4581406694563, total=  15.0s
[CV] alpha=1.701254279852589 .........................................
[CV]  alpha=1.425102670302998, score=-1660.0265052118414, total=  17.7s
[CV] alpha=1.701254279852589 .........................................
[CV]  alpha=1.701254279852589, score=-1741.372633919039, total=  15.8s
[CV] alpha=1.701254279852589 .........................................
[CV]  alpha=1.701254279852589, score=-1746.081328001392, total=  15.3s
[CV] alpha=2.0309176209047357 ........................................


...

[CV]  alpha=412.4626382901352, score=-1656.8323131545392, total=  11.0s
[CV] alpha=492.3882631706742 .........................................
[CV]  alpha=412.4626382901352, score=-1769.9559840445395, total=  10.7s
[CV] alpha=492.3882631706742 .........................................
[CV]  alpha=492.3882631706742, score=-1738.810682076626, total=   9.8s
[CV] alpha=492.3882631706742 .........................................
[CV]  alpha=492.3882631706742, score=-1743.735844241921, total=   8.4s
[CV] alpha=587.8016072274912 .........................................
[CV]  alpha=492.3882631706742, score=-1759.9697022739174, total=   7.7s
[CV] alpha=587.8016072274912 .........................................
[CV]  alpha=492.3882631706742, score=-1656.8520946416127, total=   7.1s
[CV] alpha=587.8016072274912 .........................................
[CV]  alpha=492.3882631706742, score=-1770.0311927616228, total=   7.0s
[CV] alpha=587.8016072274912 .........................................
[

[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  8.1min


[CV]  alpha=837.6776400682925, score=-1760.1559255485377, total=   6.9s
[CV] alpha=1000.0 ....................................................
[CV]  alpha=837.6776400682925, score=-1657.0277567019732, total=   6.9s
[CV] alpha=1000.0 ....................................................
[CV]  alpha=837.6776400682925, score=-1770.3863923830131, total=   5.5s
[CV] alpha=1000.0 ....................................................
[CV] ........... alpha=1000.0, score=-1739.070949846634, total=   5.2s
[CV] alpha=1000.0 ....................................................
[CV] .......... alpha=1000.0, score=-1744.0024554641438, total=   5.0s
[CV] .......... alpha=1000.0, score=-1760.2611977588517, total=   4.6s
[CV] .......... alpha=1000.0, score=-1770.5568402013191, total=   3.4s
[CV] .......... alpha=1000.0, score=-1657.1321726361016, total=   4.0s


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  8.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': array([   1.     ,    1.19378,    1.4251 ,    1.70125,    2.03092,
          2.42446,    2.89427,    3.45511,    4.12463,    4.92388,
          5.87802,    7.01704,    8.37678,   10.     ,   11.93777,
         14.25103,   17.01254,   20.30918,   24.24462,   28.94266,
         34....42661,  345.51073,  412.46264,
        492.38826,  587.80161,  701.70383,  837.67764, 1000.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=10)

In [354]:
print(ridge.best_params_)

{'alpha': 289.4266124716752}


In [370]:
# Prédiction sur le jeu de tess
y_pred_ridge = ridge.predict(X_test)

# Evaluate
print("RMSE : %.2f" % np.sqrt(metrics.mean_squared_error(y_test, y_pred_ridge)))

RMSE : 41.81


Avec une regression Ridge, la performance est ici meilleur avec un RMSE de 41,81. Essayons maintenant la regression lasso. 

## Lasso

In [356]:
lasso = model_selection.GridSearchCV(Lasso(), cv=5,
                                      param_grid={"alpha": np.logspace(-2.5, 0, 40)},
                                    verbose=10, n_jobs=-1, scoring='neg_mean_squared_error')


In [357]:
lasso.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] alpha=0.0031622776601683794 .....................................
[CV] alpha=0.0031622776601683794 .....................................
[CV] alpha=0.0031622776601683794 .....................................
[CV] alpha=0.0031622776601683794 .....................................
[CV]  alpha=0.0031622776601683794, score=-1738.909255212816, total= 5.1min
[CV] alpha=0.0031622776601683794 .....................................
[CV]  alpha=0.0031622776601683794, score=-1744.071871698372, total= 5.1min
[CV] alpha=0.003665241237079626 ......................................
[CV]  alpha=0.0031622776601683794, score=-1760.4118193370928, total= 5.4min
[CV] alpha=0.003665241237079626 ......................................
[CV]  alpha=0.0031622776601683794, score=-1657.066660697359, total= 5.5min
[CV] alpha=0.003665241237079626 ......................................
[CV]  alpha=0.003665241237079626, score=-1738.9266732470417, total= 3

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  9.0min


[CV]  alpha=0.003665241237079626, score=-1744.151953004442, total= 3.8min
[CV] alpha=0.003665241237079626 ......................................
[CV]  alpha=0.003665241237079626, score=-1760.4521831005793, total= 4.2min
[CV] alpha=0.004248201698162612 ......................................
[CV]  alpha=0.0031622776601683794, score=-1770.2594337224925, total= 5.1min
[CV] alpha=0.004248201698162612 ......................................
[CV]  alpha=0.004248201698162612, score=-1738.9505047136633, total= 3.6min
[CV] alpha=0.004248201698162612 ......................................
[CV]  alpha=0.003665241237079626, score=-1657.0892439864697, total= 4.5min
[CV] alpha=0.004248201698162612 ......................................


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 13.5min


[CV]  alpha=0.004248201698162612, score=-1744.2817380730698, total= 3.7min
[CV] alpha=0.004248201698162612 ......................................
[CV]  alpha=0.003665241237079626, score=-1770.2958117537837, total= 4.7min
[CV] alpha=0.004923882631706742 ......................................
[CV]  alpha=0.004923882631706742, score=-1739.002410242683, total= 3.3min
[CV] alpha=0.004923882631706742 ......................................
[CV]  alpha=0.004248201698162612, score=-1760.4926543679446, total= 4.4min
[CV] alpha=0.004923882631706742 ......................................
[CV]  alpha=0.004248201698162612, score=-1657.1710444062426, total= 4.3min
[CV] alpha=0.004923882631706742 ......................................
[CV]  alpha=0.004248201698162612, score=-1770.3594082978134, total= 4.3min
[CV] alpha=0.004923882631706742 ......................................
[CV]  alpha=0.004923882631706742, score=-1744.3563118350844, total= 3.1min
[CV] alpha=0.005707031326057167 ..................

...

[CV] alpha=0.5541020330009492 ........................................
[CV]  alpha=0.5541020330009492, score=-1798.7416900549003, total=   3.9s
[CV] alpha=0.5541020330009492 ........................................
[CV]  alpha=0.47806525330073807, score=-1830.6429191249542, total=   4.7s
[CV] alpha=0.5541020330009492 ........................................
[CV]  alpha=0.5541020330009492, score=-1796.8424549166245, total=   3.6s
[CV] alpha=0.6422325422229356 ........................................
[CV]  alpha=0.5541020330009492, score=-1821.8489929690359, total=   3.8s
[CV] alpha=0.6422325422229356 ........................................
[CV]  alpha=0.5541020330009492, score=-1715.3164107705443, total=   4.0s
[CV]  alpha=0.5541020330009492, score=-1833.8175728221017, total=   3.9s
[CV] alpha=0.6422325422229356 ........................................
[CV] alpha=0.6422325422229356 ........................................
[CV]  alpha=0.6422325422229356, score=-1801.1419017909789, total

[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 52.8min


[CV]  alpha=0.8627747685955867, score=-1824.9157948163459, total=   1.6s
[CV] alpha=1.0 .......................................................
[CV]  alpha=0.8627747685955867, score=-1718.5781530282738, total=   1.8s
[CV] alpha=1.0 .......................................................
[CV]  alpha=0.8627747685955867, score=-1836.9009433979772, total=   1.9s
[CV] .............. alpha=1.0, score=-1802.167830408348, total=   1.8s
[CV] alpha=1.0 .......................................................
[CV] alpha=1.0 .......................................................
[CV] ............. alpha=1.0, score=-1800.4635585651288, total=   1.6s
[CV] ............. alpha=1.0, score=-1824.9157948163459, total=   1.6s
[CV] ............. alpha=1.0, score=-1718.5781530282738, total=   1.6s
[CV] ............. alpha=1.0, score=-1836.9009433979772, total=   1.5s


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 52.9min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': array([0.00316, 0.00367, 0.00425, 0.00492, 0.00571, 0.00661, 0.00767,
       0.00889, 0.0103 , 0.01194, 0.01384, 0.01604, 0.01859, 0.02154,
       0.02497, 0.02894, 0.03355, 0.03888, 0.04507, 0.05223, 0.06054,
       0.07017, 0.08133, 0.09427, 0.10926, 0.12664, 0.14678, 0.17013,
       0.19718, 0.22855, 0.2649 , 0.30703, 0.35586, 0.41246, 0.47807,
       0.5541 , 0.64223, 0.74438, 0.86277, 1.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=10)

In [363]:
print(lasso.best_params_)

{'alpha': 0.0031622776601683794}


In [371]:
# Prédiction sur le jeu de test
y_pred_lasso = lasso.predict(X_test)

# Evaluate
print("RMSE : %.2f" % np.sqrt(metrics.mean_squared_error(y_test, y_pred_lasso)))

RMSE : 41.81


On trouve ici le même RMSE ce qui nous donne une performance similaire pour les 2 modèles. 

## Modèle final : régression Ridge

Le modèle final choisi est donc une regression Ridge car avec le lasso ils ont le RMSE le plus bas : 41,81 et en effectuant ces tests sur la totalité de la base de données, la regression Ridge était plus performante.   